In [167]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [171]:
# Load the csv exported in part I to a DataFrame

humidity_df = pd.read_csv('output_data_cities.csv')
humidity_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Humidity,Date
0,ribeira grande,PT,38.5167,-28.7000,62.60,16.11,75,94,1619912238
1,gunupur,IN,19.0833,83.8167,76.96,0.34,2,83,1619912517
2,rikitea,PF,-23.1203,-134.9692,75.67,8.52,93,80,1619911979
3,isangel,VU,-19.5500,169.2667,78.80,9.22,90,78,1619912520
4,geraldton,AU,-28.7667,114.6000,50.00,6.91,72,66,1619912521
...,...,...,...,...,...,...,...,...,...
548,sailana,IN,23.4667,74.9167,80.92,7.14,93,34,1619913321
549,ketchikan,US,55.3422,-131.6461,46.60,3.29,96,91,1619913322
550,mungwi,ZM,-10.1732,31.3694,61.86,3.91,95,96,1619913323
551,ponta delgada,PT,37.7333,-25.6667,59.00,14.97,75,100,1619913325


# Humidity Heatmap

In [172]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = humidity_df[['Lat','Lng']]
humidity = humidity_df['Humidity']

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [173]:
# Create new DataFrame fitting weather criteria
hotel_df = pd.DataFrame(humidity_df, columns = ['City', 'Country','Lat','Lng', 'Max Temp', 'Wind Speed', 'Humidity'])

# Narrow down the cities to fit weather conditions
max_temp = (hotel_df['Max Temp'] > 39) & (humidity_df['Max Temp'] < 76)
wind = hotel_df['Wind Speed'] < 9  
humidity = hotel_df['Humidity'] < 76

# Drop any rows with null values
hotel_df.dropna()

hotel_df[max_temp & wind & humidity]

hotel_df


,City,Country,Lat,Lng,Max Temp,Wind Speed,Humidity
0,ribeira grande,PT,38.5167,-28.7000,62.60,16.11,94
1,gunupur,IN,19.0833,83.8167,76.96,0.34,83
2,rikitea,PF,-23.1203,-134.9692,75.67,8.52,80
3,isangel,VU,-19.5500,169.2667,78.80,9.22,78
4,geraldton,AU,-28.7667,114.6000,50.00,6.91,66
...,...,...,...,...,...,...,...
548,sailana,IN,23.4667,74.9167,80.92,7.14,34
549,ketchikan,US,55.3422,-131.6461,46.60,3.29,91
550,mungwi,ZM,-10.1732,31.3694,61.86,3.91,96
551,ponta delgada,PT,37.7333,-25.6667,59.00,14.97,100


# Hotel Map

In [176]:
# Store into variable named hotel_df.
hotel_df['Hotel Name'] = ''

# Add a "Hotel Name" column to the DataFrame.
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Humidity,Hotel Name
0,ribeira grande,PT,38.5167,-28.7000,62.60,16.11,94,
1,gunupur,IN,19.0833,83.8167,76.96,0.34,83,
2,rikitea,PF,-23.1203,-134.9692,75.67,8.52,80,
3,isangel,VU,-19.5500,169.2667,78.80,9.22,78,
4,geraldton,AU,-28.7667,114.6000,50.00,6.91,66,
...,...,...,...,...,...,...,...,...
548,sailana,IN,23.4667,74.9167,80.92,7.14,34,
549,ketchikan,US,55.3422,-131.6461,46.60,3.29,91,
550,mungwi,ZM,-10.1732,31.3694,61.86,3.91,96,
551,ponta delgada,PT,37.7333,-25.6667,59.00,14.97,100,


In [177]:
# Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {'type': 'hotel',
          'radius': 5000,
          'key': g_key}

# Iterate through rows
for index, row in hotel_df.iterrows():
    # Lattitude and longitutde each city
    lat = row['Lat']
    lng = row['Lng']
    
    # Add location parameter
    params['location'] = f'{lat},{lng}'

    # API request
    print(f'Retrieving Results for Index {index}: {city_name}.')
    response = requests.get(base_url, params=params).json()
    
    # Pull results
    results = response['results']
    
    # Try-except clause
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except:
        print('Missing field/result... skipping.')
        
print('------------------------')
print('Data Retrieval Complete')
print('------------------------')

Retrieving Results for Index 0: chokurdakh.
Closest hotel in chokurdakh is Horta.
Retrieving Results for Index 1: chokurdakh.
Closest hotel in chokurdakh is Gunupur.
Retrieving Results for Index 2: chokurdakh.
Closest hotel in chokurdakh is Rikitea.
Retrieving Results for Index 3: chokurdakh.
Closest hotel in chokurdakh is Isangel.
Retrieving Results for Index 4: chokurdakh.
Closest hotel in chokurdakh is Geraldton.
Retrieving Results for Index 5: chokurdakh.
Closest hotel in chokurdakh is Kodiak.
Retrieving Results for Index 6: chokurdakh.
Closest hotel in chokurdakh is Bluff.
Retrieving Results for Index 7: chokurdakh.
Closest hotel in chokurdakh is Atuona.
Retrieving Results for Index 8: chokurdakh.
Closest hotel in chokurdakh is Tiksi.
Retrieving Results for Index 9: chokurdakh.
Closest hotel in chokurdakh is Constitución.
Retrieving Results for Index 10: chokurdakh.
Closest hotel in chokurdakh is Hobart.
Retrieving Results for Index 11: chokurdakh.
Closest hotel in chokurdakh is S

Closest hotel in chokurdakh is Namibe.
Retrieving Results for Index 96: chokurdakh.
Closest hotel in chokurdakh is Hostotipaquillo.
Retrieving Results for Index 97: chokurdakh.
Missing field/result... skipping.
Retrieving Results for Index 98: chokurdakh.
Closest hotel in chokurdakh is Höfn.
Retrieving Results for Index 99: chokurdakh.
Missing field/result... skipping.
Retrieving Results for Index 100: chokurdakh.
Closest hotel in chokurdakh is Cururupu.
Retrieving Results for Index 101: chokurdakh.
Closest hotel in chokurdakh is Mayo.
Retrieving Results for Index 102: chokurdakh.
Closest hotel in chokurdakh is Jardim.
Retrieving Results for Index 103: chokurdakh.
Closest hotel in chokurdakh is Port Macquarie.
Retrieving Results for Index 104: chokurdakh.
Closest hotel in chokurdakh is Half Moon Bay.
Retrieving Results for Index 105: chokurdakh.
Closest hotel in chokurdakh is New Norfolk.
Retrieving Results for Index 106: chokurdakh.
Closest hotel in chokurdakh is Punta Arenas.
Retriev

Closest hotel in chokurdakh is Tazovsky.
Retrieving Results for Index 190: chokurdakh.
Closest hotel in chokurdakh is Port Lincoln.
Retrieving Results for Index 191: chokurdakh.
Closest hotel in chokurdakh is General Roca.
Retrieving Results for Index 192: chokurdakh.
Closest hotel in chokurdakh is Coimbra.
Retrieving Results for Index 193: chokurdakh.
Closest hotel in chokurdakh is St. John's.
Retrieving Results for Index 194: chokurdakh.
Closest hotel in chokurdakh is Poddorye.
Retrieving Results for Index 195: chokurdakh.
Closest hotel in chokurdakh is Provideniya.
Retrieving Results for Index 196: chokurdakh.
Closest hotel in chokurdakh is Arohane.
Retrieving Results for Index 197: chokurdakh.
Closest hotel in chokurdakh is Malinovoe Ozero.
Retrieving Results for Index 198: chokurdakh.
Closest hotel in chokurdakh is Pu'er City.
Retrieving Results for Index 199: chokurdakh.
Closest hotel in chokurdakh is Kenora.
Retrieving Results for Index 200: chokurdakh.
Closest hotel in chokurda

Closest hotel in chokurdakh is Neftchala.
Retrieving Results for Index 284: chokurdakh.
Closest hotel in chokurdakh is Jiuquan.
Retrieving Results for Index 285: chokurdakh.
Closest hotel in chokurdakh is Grand Anse.
Retrieving Results for Index 286: chokurdakh.
Closest hotel in chokurdakh is SMK Negeri 1 Kalianget.
Retrieving Results for Index 287: chokurdakh.
Closest hotel in chokurdakh is Plomari.
Retrieving Results for Index 288: chokurdakh.
Closest hotel in chokurdakh is Paoua.
Retrieving Results for Index 289: chokurdakh.
Missing field/result... skipping.
Retrieving Results for Index 290: chokurdakh.
Closest hotel in chokurdakh is Klaksvík.
Retrieving Results for Index 291: chokurdakh.
Closest hotel in chokurdakh is Santa Cruz de la Sierra.
Retrieving Results for Index 292: chokurdakh.
Closest hotel in chokurdakh is Juneau.
Retrieving Results for Index 293: chokurdakh.
Closest hotel in chokurdakh is Philipsburg.
Retrieving Results for Index 294: chokurdakh.
Closest hotel in choku

Closest hotel in chokurdakh is Sydney.
Retrieving Results for Index 377: chokurdakh.
Closest hotel in chokurdakh is Bonavista.
Retrieving Results for Index 378: chokurdakh.
Closest hotel in chokurdakh is Oodweyne.
Retrieving Results for Index 379: chokurdakh.
Closest hotel in chokurdakh is Mareeba.
Retrieving Results for Index 380: chokurdakh.
Closest hotel in chokurdakh is Ballina.
Retrieving Results for Index 381: chokurdakh.
Closest hotel in chokurdakh is Nizhny Kuranakh.
Retrieving Results for Index 382: chokurdakh.
Closest hotel in chokurdakh is Am Timan.
Retrieving Results for Index 383: chokurdakh.
Closest hotel in chokurdakh is Rafaela.
Retrieving Results for Index 384: chokurdakh.
Closest hotel in chokurdakh is Cabedelo.
Retrieving Results for Index 385: chokurdakh.
Closest hotel in chokurdakh is Sabrang.
Retrieving Results for Index 386: chokurdakh.
Closest hotel in chokurdakh is Caravelas.
Retrieving Results for Index 387: chokurdakh.
Closest hotel in chokurdakh is Måløy.
Re

Closest hotel in chokurdakh is Naha.
Retrieving Results for Index 471: chokurdakh.
Closest hotel in chokurdakh is Kupang.
Retrieving Results for Index 472: chokurdakh.
Closest hotel in chokurdakh is Eyl.
Retrieving Results for Index 473: chokurdakh.
Closest hotel in chokurdakh is Oriximiná.
Retrieving Results for Index 474: chokurdakh.
Closest hotel in chokurdakh is Montijo.
Retrieving Results for Index 475: chokurdakh.
Closest hotel in chokurdakh is Chuy.
Retrieving Results for Index 476: chokurdakh.
Closest hotel in chokurdakh is Ekhabi.
Retrieving Results for Index 477: chokurdakh.
Closest hotel in chokurdakh is Yancheng.
Retrieving Results for Index 478: chokurdakh.
Closest hotel in chokurdakh is Ürümqi.
Retrieving Results for Index 479: chokurdakh.
Closest hotel in chokurdakh is Tonantins.
Retrieving Results for Index 480: chokurdakh.
Closest hotel in chokurdakh is Nicoya.
Retrieving Results for Index 481: chokurdakh.
Closest hotel in chokurdakh is Robertsport.
Retrieving Results 

In [178]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Humidity,Hotel Name
0,ribeira grande,PT,38.5167,-28.7000,62.60,16.11,94,Horta
1,gunupur,IN,19.0833,83.8167,76.96,0.34,83,Gunupur
2,rikitea,PF,-23.1203,-134.9692,75.67,8.52,80,Rikitea
3,isangel,VU,-19.5500,169.2667,78.80,9.22,78,Isangel
4,geraldton,AU,-28.7667,114.6000,50.00,6.91,66,Geraldton
...,...,...,...,...,...,...,...,...
548,sailana,IN,23.4667,74.9167,80.92,7.14,34,Sailana
549,ketchikan,US,55.3422,-131.6461,46.60,3.29,91,Ketchikan
550,mungwi,ZM,-10.1732,31.3694,61.86,3.91,96,Mungwi
551,ponta delgada,PT,37.7333,-25.6667,59.00,14.97,100,Ponta Delgada


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [143]:
# Add marker layer on top of heat map


# Configure gmaps.
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = humidity_df[['Lat','Lng']]
humidity = humidity_df['Humidity']

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add Heatmap layer to map.
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…